This block is responsible for splitting the traces generated by kieker corresponding to each business capability according to the test tag sent while executing the test cases.

In [ ]:
import os
import re
from pathlib import Path
from collections import defaultdict

def split_kieker_dat_files(input_dir):
    input_path = Path(input_dir)
    dat_files = sorted(input_path.glob("*.dat"))
    
    test_case_pattern = re.compile(r'\$2;\d+;Test_([^_]+)_([^\s]+)')
    
    for dat_file in dat_files:
        with open(dat_file, 'r') as file:
            lines = file.readlines()
        
        current_lines = []
        current_business = None
        counters = defaultdict(int)

        for line in lines:
            match = test_case_pattern.match(line)
            if match:
                business, test_case = match.groups()
                if current_lines:
                    write_test_case_file(input_path, dat_file.name, current_business, counters[current_business], current_lines)
                
                current_business = business
                counters[business] += 1
                current_lines = [line]
            else:
                if current_business:
                    current_lines.append(line)
        
        if current_lines and current_business:
            write_test_case_file(input_path, dat_file.name, current_business, counters[current_business], current_lines)

def write_test_case_file(root_path, original_filename, business_name, counter, lines):
    business_dir = root_path / business_name
    business_dir.mkdir(exist_ok=True)
    
    file_stem = "-".join(original_filename.split("-")[:-1])
    new_filename = f"{file_stem}-{counter:03d}.dat"
    new_file_path = business_dir / new_filename
    
    with open(new_file_path, 'w') as f:
        f.writelines(lines)

    print(f"Created {new_file_path} with {len(lines)} lines")

def main():
    input_dir = "/Users/fabiha/Documents/mono2micro/mono2micro/petclinic/petclinic_5.3.22_kieker_instrumented/kieker_logs/kieker-20250627-021502-178206820999419-UTC--KIEKER"
    split_kieker_dat_files(input_dir)

if __name__ == "__main__":
    main()
